In [1]:
import mmcv
from mmcv import Config
from mmdet.datasets import (build_dataloader, build_dataset,
                            replace_ImageToTensor)
from mmdet.models import build_detector
from mmdet.apis import single_gpu_test
from mmcv.runner import load_checkpoint
import os
from mmcv.parallel import MMDataParallel
import pandas as pd
from pandas import DataFrame
from pycocotools.coco import COCO
import numpy as np
from mmcv import Config

from mycode.set_num_classes import set_num_classes
from mycode.set_options import set_test

# https://github.com/zFTurbo/Weighted-Boxes-Fusion
# from ensemble_boxes import *

/opt/conda/lib/python3.7/site-packages/mmcv/utils/registry.py:250: UserWarning: The old API of register_module(module, force=False) is deprecated and will be removed, please use the new API register_module(name=None, force=False, module=None) instead.
  'The old API of register_module(module, force=False) '


In [ ]:
#########################################################################################################

epoch = 18
model_name = 'detectors'
cfg_name='detectors_x101_64x4d_adam2'

checkpoint_path = os.path.join(f'./work_dirs', cfg_name, f'epoch_{epoch}.pth')
# checkpoint_path = os.path.join('./work_dirs', cfg_name, 'best_bbox_mAP_50.pth') 

##########################################################################################################

cfg = Config.fromfile(os.path.join('./configs/trash', model_name, f'{cfg_name}.py'))

set_test(cfg, cfg_name)

#########################################################################################################

dataset = build_dataset(cfg.data.test)
data_loader = build_dataloader(
        dataset,
        samples_per_gpu=1,
        workers_per_gpu=cfg.data.workers_per_gpu,
        dist=False,
        shuffle=False)

#########################################################################################################

model = build_detector(cfg.model, test_cfg=cfg.get('test_cfg'))
checkpoint = load_checkpoint(model, checkpoint_path, map_location='cpu')
model.CLASSES = dataset.CLASSES
model = MMDataParallel(model.cuda(), device_ids=[0])

#########################################################################################################

output = single_gpu_test(model, data_loader, show_score_thr=0.05)

#########################################################################################################

prediction_strings = []
file_names = []
coco = COCO(cfg.data.test.ann_file)
imag_ids = coco.getImgIds()

class_num = 11
for i, out in enumerate(output):
    prediction_string = ''
    image_info = coco.loadImgs(coco.getImgIds(imgIds=i))[0]
    for j in range(class_num):
        for o in out[j]:
            prediction_string += str(j) + ' ' + str(o[4]) + ' ' + str(o[0]) + ' ' + str(o[1]) + ' ' + str(
                o[2]) + ' ' + str(o[3]) + ' '

    prediction_strings.append(prediction_string)
    file_names.append(image_info['file_name'])


submission = pd.DataFrame()
submission['PredictionString'] = prediction_strings
submission['image_id'] = file_names
submission.to_csv(os.path.join('./work_dirs', cfg_name, cfg_name+f'_epoch{epoch}'+'.csv'), index=None)
submission.head()

#############################################################################################################